In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException

from webdriver_auto_update import check_driver

from typing import List

import pathlib
import time


In [2]:
G_CURRENT_DIR = pathlib.Path().resolve()
G_DOWNLOAD_DIR = f"{G_CURRENT_DIR}\download"
G_CURRENT_DIR
f"{G_CURRENT_DIR}\\chromedriver.exe"

'C:\\dev\\proj\\grammar-kabardian\\subproj\\scrape_amaltus_py\\chromedriver.exe'

In [3]:

G_CURRENT_DIR = pathlib.Path().resolve()
G_DOWNLOAD_DIR = f"{G_CURRENT_DIR}\download"


def get_webdriver() -> webdriver.Chrome:
    '''
    Returns a webdriver.Chrome object.
    '''
    check_driver("")

    driver_path = f"{G_CURRENT_DIR}\chromedriver.exe"

    options = webdriver.ChromeOptions()
    dpref = {
        "download.default_directory": G_DOWNLOAD_DIR
    }
    options.add_experimental_option('prefs', dpref)
    # options.add_argument("headless")
    options.add_argument("--enable-javascript")
    # options.add_argument("--disable-gpu")
    service = Service(executable_path=driver_path)
    return webdriver.Chrome(service=service, options=options)


def handle_entry(driver: webdriver.Chrome) -> list[str]:
    '''
    Work horse of the program.

    '''

    # print("------------------------------\nhandle_entry:\n---\n\n")
    wdw = WebDriverWait(driver, 1)
    xpath_base = "//*[@id='fragment-3']"
    xpath_list_elem = "//div[@id='fragment-3']/li"
    # //*[@id="fragment-3"]/li/text()[2]

    try:
        wdw.until(EC.presence_of_element_located((By.XPATH, xpath_base)))
    except Exception as e:
        print("EXCEPTION 0: ", e)
        return []

    list_elem: WebElement = driver.find_element(By.XPATH, xpath_list_elem)

    nodes = driver.execute_script(
        "return arguments[0].childNodes", list_elem)
    text_nodes: list[str] = []

    for node in nodes:
        if isinstance(node, str):
            _text = node['textContent'].strip()
            if _text:
                text_nodes.append(_text)
        elif isinstance(node, dict):
            _text = node['textContent'].strip()
            if _text:
                text_nodes.append(_text)
        elif isinstance(node, WebElement):
            node: WebElement
            text_nodes.append(f"<{node.tag_name}>{node.text}")
        else:
            err_text = f"EXCEPTION 123.\n"
            f"Type: {type(node)}\n"
            f"Content: {node}\n"
            f"node.keys: {node.keys()}\n"
            f"node.text: {node['textContent']}\n"
            print(err_text)
            raise Exception(err_text)
    # print(text_nodes)
    return text_nodes


def handle_entry_block(driver: webdriver.Chrome) -> list[list[str]]:
    xpath_elem = "//*[@id='fragment-2']/li/ul"

    try:
        elem_list = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, xpath_elem))
        )
    except TimeoutException as e:
        print("Element not found within time limit")
        elem_list = None
    # elem_list = driver.find_element(By.XPATH, xpath_elem)
    elems = elem_list.find_elements(By.TAG_NAME, "li")
    # print(len(elems)) # max 20
    entries: list[list[str]] = []
    for e in elems:
        entry_elem = e.find_element(By.TAG_NAME, "a")
        entry_elem.click()
        entry = handle_entry(driver)

        driver.find_element(
            By.XPATH, "//*[@id='tabs']/ul/li[2]/a").click()
        entries.append(entry)
    return entries


In [4]:
G_DRIVER: webdriver.Chrome = get_webdriver()


def add_entries_to_file(file_name: str, entries: list[list[str]]) -> None:
    '''
    Appends a list of entries to a file. Each entry is written to a new line.
    The entries are stored in the format of a list of strings.
    '''
    with open(file_name, "a", encoding="utf-8") as f:
        for entry in entries:
            f.write(str(entry) + "\n")


amaltus_kbd_to_rus = "http://amaltus.com/%d0%ba%d0%b0%d0%b1%d0%b0%d1%80%d0%b4%d0%b8%d0%bd%d0%be-%d1%80%d1%83%d1%81%d1%81%d0%ba%d0%b8%d0%b9-%d1%81%d0%bb%d0%be%d0%b2%d0%b0%d1%80%d1%8c/"
G_DRIVER.get(amaltus_kbd_to_rus)

wdw = WebDriverWait(G_DRIVER, 1)
xpath_button_sb = "//*[@id='sb_butt']"

# Here we are in the main search page, and we click on the search button without any input
# so that we get the whole list of words
try:
    button_sb = wdw.until(EC.element_to_be_clickable(
        (By.XPATH, xpath_button_sb)))
    button_sb.click()

except NoSuchElementException as e:
    print(e)

    ##############################################

    # Here we are now in the list of words.s
    # We go through each of them


xpath_base = "//*[@id='fragment-2']/li/ul/li[1]/a"
xpath_elem = "//*[@id='fragment-2']/li/ul"
try:
    wdw.until(EC.element_to_be_clickable(
        (By.XPATH, xpath_elem)))
except NoSuchElementException as e:
    print(e)

try:

    while True:
        entries = handle_entry_block(G_DRIVER)
        add_entries_to_file("entries.txt", entries)

        G_DRIVER.find_element(By.XPATH, "//*[@id='next_nav']").click()
        time.sleep(0.5)

except NoSuchElementException as e:
    print(e)
time.sleep(5)
G_DRIVER.close()


Local chromedriver version: 114.0.5735.90
Latest online chromedriver version: 114.0.5735.90
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='next_nav']"}
  (Session info: chrome=114.0.5735.198)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0087A813+48355]
	(No symbol) [0x0080C4B1]
	(No symbol) [0x00715358]
	(No symbol) [0x007409A5]
	(No symbol) [0x00740B3B]
	(No symbol) [0x0076E232]
	(No symbol) [0x0075A784]
	(No symbol) [0x0076C922]
	(No symbol) [0x0075A536]
	(No symbol) [0x007382DC]
	(No symbol) [0x007393DD]
	GetHandleVerifier [0x00ADAABD+2539405]
	GetHandleVerifier [0x00B1A78F+2800735]
	GetHandleVerifier [0x00B1456C+2775612]
	GetHandleVerifier [0x009051E0+616112]
	(No symbol) [0x00815F8C]
	(No symbol) [0x00812328]
	(No symbol) [0x0081240B]
	(No symbol) [0x00804FF7]
	BaseThreadInitThunk [0x76897D59+25]
	RtlInitializeExceptionChain [0x778EB74B+107]
	RtlClearBits [0x778EB6CF+191]



In [ ]:
button_sb